In [1]:
from langchain_openai import OpenAIEmbeddings
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import os

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up connections

In [2]:
openai_api_key = getpass()

 ········


In [3]:
neo4j_password = getpass()

 ········


In [4]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key)

# Embed topic descriptions

In [6]:
topics = gds.run_cypher("""
                        MATCH (t:Topic)
                        WHERE t.embedding IS NULL
                        RETURN t.description AS description""")

In [7]:
topics['embedding'] = embeddings.embed_documents(topics['description'].to_list())

In [8]:
topics.head()

,description,embedding
0,,"[0.015375844493788012, -0.034777941596249584, ..."
1,! — NOT expression in Cypher,"[-0.001502172470261095, 0.03528362585172272, 0..."
2,!RATED relationship,"[0.01292134665386383, 0.02400438344185642, -0...."
3,!tables command,"[0.0033105522452637954, 0.0042203221288704, 0...."
4,"""(RELATIONSHIP)"" suffix","[0.019056276849615902, 0.0011400599897678637, ..."


In [9]:
gds.run_cypher("""CREATE VECTOR INDEX topic_vectors IF NOT EXISTS 
                  FOR (t:Topic)
                  ON (t.embedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 1536,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [12]:
for i in range(0, int(topics.shape[0]/1000) + 1):
    gds.run_cypher("""
        UNWIND $topicData AS d
        MATCH (t:Topic {description:trim(d['description'])})
        CALL db.create.setNodeVectorProperty(t, 'embedding', d['embedding'])""",
                   {'topicData': topics.iloc[i*1000:(i+1)*1000,:].to_dict('records')})
    if (i+1)%5 == 0:
        print(f"Finished row {(i+1)*1000}")
    

Finished row 5000
Finished row 10000
Finished row 15000
Finished row 20000
Finished row 25000
Finished row 30000
Finished row 35000
Finished row 40000
Finished row 45000
Finished row 50000
Finished row 55000
Finished row 60000
Finished row 65000
Finished row 70000


# Calculate nearest neighbors

In [92]:
g_topics, result = gds.graph.project("topics", {"Topic":{"properties": "embedding"}}, "*")

In [94]:
result

nodeProjection                        {'Topic': {'label': 'Topic', 'properties': {'embedding': {'property': 'embedding', 'defaultValue': None}}}}
relationshipProjection    {'__ALL__': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {}, 'type': '*'}}
graphName                                                                                                                                  topics
nodeCount                                                                                                                                   70954
relationshipCount                                                                                                                               0
projectMillis                                                                                                                                1775
Name: 0, dtype: object

## Check similiarty scores for some sample document pairs

In [95]:
gds.run_cypher("""
UNWIND
[
    ['Cypher MATCH clause', 'Cypher MATCH statement'],
    ['Cypher MATCH clause', 'Inline MATCH clause'],
    ['MATCH (n)-[rs*]-() syntax deprecated', 'Cypher MATCH clause'],
    ['MATCH in Cypher', 'Cypher MATCH clause'],
    ['MATCH query', 'MATCH in Cypher'],
    ['MATCH with LIMIT in Cypher', 'MATCH in Cypher']
] AS themes
MATCH (n1), (n2) WHERE n1.description = themes[0] and n2.description = themes[1] 
RETURN 
n1.description AS theme1,
n2.description AS theme2,
gds.similarity.cosine(n1.embedding, n2.embedding) AS sim""")

,theme1,theme2,sim
0,Cypher MATCH clause,Cypher MATCH statement,0.946668
1,Cypher MATCH clause,Inline MATCH clause,0.708626
2,MATCH (n)-[rs*]-() syntax deprecated,Cypher MATCH clause,0.525790
3,MATCH in Cypher,Cypher MATCH clause,0.784752
4,MATCH query,MATCH in Cypher,0.547737
5,MATCH with LIMIT in Cypher,MATCH in Cypher,0.741662


## Write KNN at a level below which we don't think there will be many meaningful pairs

In [96]:
knn_result = gds.knn.mutate(g_topics,
                            mutateRelationshipType = "IS_SIMILAR",
                            mutateProperty = "similarity",
                            nodeProperties = "embedding",
                            topK = 4,
                            similarityCutoff = 0.85)

In [97]:
similarity_result = gds.run_cypher("""
    CALL gds.graph.relationshipProperty.stream("topics", "similarity", ["IS_SIMILAR"])
    YIELD sourceNodeId, targetNodeId, propertyValue AS similarity
    WITH gds.util.asNode(sourceNodeId) AS source, gds.util.asNode(targetNodeId) AS target, similarity
    RETURN source.description AS source, target.description AS target, similarity
    ORDER BY similarity""")
                    

In [99]:
similarity_result[similarity_result['similarity']>0.87].head(20)

,source,target,similarity
27273,CypherSyntaxError exception,Cypher syntax suggestions,0.870000
27274,Data retrieval methods,Database retrieval,0.870001
27275,NODE(*) in Cypher,node in Cypher,0.870002
27276,Aura SSL connection,Configure SSL certificate,0.870005
27277,rename label in Neo4j,Neo4j label treatment,0.870006
27278,Adding Class nodes in Cypher,Connected nodes in Cypher,0.870006
27279,Standard input,Binary input,0.870007
27280,Binary input,Standard input,0.870007
27281,Write to the database,Writing results to database,0.870010
27282,Writing results to database,Write to the database,0.870010


# Check WCC at a reasonable similarity cutoff

In [100]:
knn_result['relationshipsWritten']

115479

In [101]:
knn_result['similarityDistribution']

{'min': 0.8499984741210938,
 'p5': 0.8540763854980469,
 'max': 0.9985771179199219,
 'p99': 0.9844398498535156,
 'p1': 0.8508110046386719,
 'p10': 0.8582344055175781,
 'p90': 0.9546279907226562,
 'p50': 0.8957557678222656,
 'p25': 0.8712501525878906,
 'p75': 0.9265403747558594,
 'p95': 0.9688911437988281,
 'mean': 0.9013724313667466,
 'p100': 0.9985771179199219,
 'stdDev': 0.035650031321179866}

In [102]:
gds.wcc.write(g_topics,
              writeProperty = "wccId",
              relationshipTypes = ["IS_SIMILAR"],
              relationshipWeightProperty = "similarity",
              threshold = 0.87,
              minComponentSize=2)

writeMillis                                                                                                                                                                                                                                                                                                                                                                  213
nodePropertiesWritten                                                                                                                                                                                                                                                                                                                                                      31337
componentCount                                                                                                                                                                                                                                                        

In [103]:
gds.run_cypher("""
MATCH (t:Topic)
WHERE t.wccId IS NOT NULL
WITH t.wccId AS id, collect(t.description) AS themes
RETURN id, size(themes) AS themeCount, themes
ORDER BY themeCount DESC
LIMIT 30""")

id  themeCount  \
0       0       28386   
1     203         457   
2     266         243   
3     479         172   
4     324         157   
5     120         129   
6     581         126   
7     240         124   
8     760         114   
9    2799         111   
10    659         107   
11   1831         105   
12    621         102   
13   2996          89   
14   6906          78   
15    981          65   
16   3932          65   
17   3887          59   
18    440          56   
19   2843          54   
20     45          52   
21   2229          50   
22    197          50   
23    352          48   
24   8077          48   
25  13369          46   
26    535          43   
27   5465          43   
28    262          42   
29    729          42   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Refactor similarity relationships for use with Leiden

In [104]:
gds.graph.relationships.toUndirected(g_topics, 
                                     relationship_type = "IS_SIMILAR", 
                                     mutate_relationship_type = "UNDIRECTED_SIMILAR", 
                                     aggregation = "MAX")

inputRelationships                                                                                                                                                                                                               115479
relationshipsWritten                                                                                                                                                                                                             171322
mutateMillis                                                                                                                                                                                                                          0
postProcessingMillis                                                                                                                                                                                                                  0
preProcessingMillis                                                     

In [105]:
gds.set_database("neo4j")
relationships = gds.graph.relationshipProperties.stream(g_topics, "similarity", ["UNDIRECTED_SIMILAR"])

Undirected relationships stream once in each direction. Drop the half of relationships where targetNodeId < sourceNodeId

In [106]:
relationships = relationships[relationships['sourceNodeId']<relationships['targetNodeId']]

The relationship weights are bunched between 0.8 and 1.0. Transform them so that they range from 0.0 to 1.0.

In [107]:
gds.run_cypher("""
UNWIND $rows AS row
WITH gds.util.asNode(row['sourceNodeId']) AS source,
gds.util.asNode(row['targetNodeId']) AS target,
row['propertyValue'] AS similarity
WITH gds.graph.project("reweight_topics",
source,
target,
{relationshipType: "IS_SIMILAR",
relationshipProperties: {similarity: (similarity-0.8)/0.2}},
{undirectedRelationshipTypes: ["IS_SIMILAR"]}) AS g
RETURN g.graphName AS graphName, g.relationshipCount AS relationshipCount, g.nodeCount AS nodeCount""",
               {"rows": relationships.to_dict("records")})

,graphName,relationshipCount,nodeCount
0,reweight_topics,171322,38226


In [108]:
g_topics2 = gds.graph.get("reweight_topics")

# Test Leiden at different values of gamma

In [109]:
def test_leiden_gamma(gamma):
    property_name = f"leidenGamma{gamma}"
    try:
        gds.graph.nodeProperties.drop(g_topics2, property_name)
    except:
        pass
    gds.leiden.mutate(g_topics2,
                      mutateProperty = property_name,
                      relationshipTypes = ["IS_SIMILAR"],
                      relationshipWeightProperty = "similarity",
                      gamma=gamma)
    biggest_communities = gds.run_cypher("""
        CALL gds.graph.nodeProperty.stream("reweight_topics", $propName)
        YIELD nodeId, propertyValue
        WITH propertyValue, collect(nodeId) AS ids, count(*) as themeCount
        ORDER BY themeCount desc
        LIMIT 20
        RETURN themeCount, [id in ids | gds.util.asNode(id).description] AS themes""",
                                        {"propName": property_name})
    return biggest_communities
                                           


In [110]:
gamma_tests = {gamma: test_leiden_gamma(gamma) for gamma in [4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0]}

In [111]:
for gamma, df in gamma_tests.items():
    print(f"Gamma value: {gamma}")
    print(df.head())

Gamma value: 4.0
   themeCount  \
0         895   
1         772   
2         723   
3         421   
4         366   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [112]:
pd.set_option('display.max_colwidth', None)

## Examine the largest clusters at various gamma values

In [113]:
gamma_tests[16.0]

,themeCount,themes
0,449,"[Dashes in Cypher, Inequalities in Cypher, Literal in Cypher, {1,3} in Cypher, CASE in Cypher, Using = in Cypher, Property in Cypher, WITH in Cypher, after in Cypher, < in Cypher, \ in Cypher, Cypher in Cypher, min in Cypher, done in Cypher, in Cypher, total in Cypher, > in Cypher, mean in Cypher, SET in Cypher, WHERE in Cypher, Message in Cypher, 'IN' type in Cypher, int in Cypher, IN in Cypher, GET in Cypher, Sleep in Cypher, title in Cypher, STARTS WITH in Cypher, START in Cypher, release year in Cypher, released in Cypher, short description in Cypher, description in Cypher, year of birth in Cypher, year in Cypher, acting roles in Cypher, OUT in Cypher, paused in Cypher, penalty in Cypher, patience in Cypher, best split in Cypher, GRANT in Cypher, CREATED in Cypher, Connect in Cypher, DESC in Cypher, ON in Cypher, AS COPY OF in Cypher, drop in Cypher, OUTGOING in Cypher, <FOLLOWS in Cypher, AWARD in Cypher, internal activity in Cypher, d in Cypher, month in Cypher, messages in Cypher, Long in Cypher, bfs in Cypher, list deconstruction in Cypher, grants in Cypher, Community property in Cypher, Multiplication in Cypher, Patterns in Cypher, DIRECTED in Cypher, Called in Cypher, Project in Cypher, Distance in Cypher, n.age in Cypher, _children in Cypher, _end in Cypher, Union parts in Cypher, Dictionary in Cypher, DEP in Cypher, Comments in Cypher, Second in Cypher, Float in Cypher, PROFILE in Cypher, issuer in Cypher, post-process in Cypher, Constituents in Cypher, ONLINE in Cypher, direction in Cypher, Hour in Cypher, minute in Cypher, second in Cypher, 'minute' in Cypher, Help message in Cypher, long in Cypher, float in Cypher, short in Cypher, setStatus in Cypher, EDGE in Cypher, [:FOLLOWS] in Cypher, relationshipsWritten in Cypher, ""ACTED_IN in Cypher"", subexpression in Cypher, Deprecations in Cypher, IS NOT :: in Cypher, (:B) in Cypher, Either direction in Cypher, Cards in Cypher, ...]"
1,413,"[scaling Neo4j, Technology Partners of Neo4j, Upgrade Neo4j, System requirements for Neo4j, Upgrading Neo4j, Naming Neo4j indexes, expectations of Neo4j, Contact Neo4j representative, Configure Neo4j, Upgrade Neo4j Community, Import Neo4j, Dump neo4j, Install Neo4j plugins, Package as Neo4j plugin, Install Neo4j adapter, Running Neo4j, Verify Neo4j installation, Default Neo4j installation, Neo4j Installation, Install Neo4j on Linux, Adding Neo4j to sources.list, Installing Neo4j, Install Neo4j locally, Starting Neo4j, Monitoring Neo4j, Install Neo4j Desktop, Contact Neo4j, Download Neo4j tools, Instantiating from Neo4j driver, Force Neo4j query solving, Investigating Neo4j problems, run Neo4j, Install single Neo4j server, Launch configuration in Neo4j, Blogging on Neo4j, Support for Neo4j, Overview of Neo4j, Learning resources for Neo4j, Get Started Free with Neo4j, Restart Neo4j, Scrape Neo4j documentation, Self-managed Neo4j, Connecting with Neo4j applications, Install Neo4j, Install Neo4j as root, configure Neo4j persistence, Unofficial Neo4j resources, Global components of Neo4j, Get Started with Neo4j for Free, General topics on Neo4j, Ingesting data into Neo4j, Accessing Neo4j externally, Free Neo4j start, Aura Neo4j+ssc, Stop Neo4j, Start Neo4j command, Native neo4j user, Resources on Neo4j, Testing instance of Neo4j, Start Neo4j, Starting with Neo4j, Introduction to Neo4j, Book chapters on Neo4j, Populate Neo4j credentials, Download from Neo4j Download Center, Compatibility with Neo4j versions, Dynamic Neo4j loaders, Release process for Neo4j, Live Neo4j sessions, Familiarity with Neo4j, Intro to Neo4j, History in Neo4j, Performance testing in Neo4j, Live Neo4j channel, Suite in Neo4j, Practitioner's view on Neo4j, General intro to Neo4j, Sharing Neo4j content, Classes in Neo4j, Business value of Neo4j, New developer blog for Neo4j, Going meta with Neo4j, Fresh Neo4j instance, Understanding Neo4j, Set of Neo4j classes, Open source Neo4j project, Community engagement with Neo4j, Online information 

In [114]:
gamma_tests[32.0]

,themeCount,themes
0,321,"[Cypher declarative nature, Dashes in Cypher, Inequalities in Cypher, Cypher analysis, Cypher in Cypher, done in Cypher, in Cypher, total in Cypher, > in Cypher, mean in Cypher, SET in Cypher, WHERE in Cypher, Message in Cypher, 'IN' type in Cypher, IN in Cypher, Sleep in Cypher, title in Cypher, STARTS WITH in Cypher, START in Cypher, release year in Cypher, released in Cypher, short description in Cypher, description in Cypher, acting roles in Cypher, OUT in Cypher, paused in Cypher, penalty in Cypher, patience in Cypher, best split in Cypher, GRANT in Cypher, CREATED in Cypher, Connect in Cypher, drop in Cypher, OUTGOING in Cypher, <FOLLOWS in Cypher, AWARD in Cypher, internal activity in Cypher, month in Cypher, messages in Cypher, Long in Cypher, bfs in Cypher, list deconstruction in Cypher, grants in Cypher, Community property in Cypher, Multiplication in Cypher, Patterns in Cypher, DIRECTED in Cypher, ACTED_IN in Cypher, Called in Cypher, Project in Cypher, Distance in Cypher, n.age in Cypher, _children in Cypher, Union parts in Cypher, Dictionary in Cypher, DEP in Cypher, Comments in Cypher, Second in Cypher, Float in Cypher, PROFILE in Cypher, issuer in Cypher, Constituents in Cypher, ONLINE in Cypher, Hour in Cypher, minute in Cypher, second in Cypher, 'minute' in Cypher, Help message in Cypher, float in Cypher, short in Cypher, EDGE in Cypher, relationshipsWritten in Cypher, ""ACTED_IN in Cypher"", subexpression in Cypher, Deprecations in Cypher, (:B) in Cypher, Cards in Cypher, zoneid in Cypher, REPLIED> in Cypher, POSTED in Cypher, REPLIED in Cypher, success in Cypher, job type in Cypher, job execution start in Cypher, quantification in Cypher, installed in Cypher, :Product in Cypher, :Order in Cypher, :SUPPLIES in Cypher, :ORDERS in Cypher, chown in Cypher, costs in Cypher, ASCENDING in Cypher, DESCENDING in Cypher, division in Cypher, inequality in Cypher, less than in Cypher, greater than in Cypher, ceil in Cypher, collect in Cypher, ...]"
1,187,"[Examples in Neo4j, scaling Neo4j, Team collaboration with Neo4j, Naming Neo4j indexes, Write results to Neo4j, Using Neo4j, score writing in Neo4j, using Neo4j, Dump neo4j, Followers in Neo4j, Graph neo4j, Friend relationships in Neo4j, run Neo4j, Default db neo4j, Ingesting data into Neo4j, History in Neo4j, Suite in Neo4j, Going meta with Neo4j, working with Neo4j, Username ""neo4j"", Unique neo4j.name values, Unified view with Neo4j, Mapping with Neo4j, Subsets in Neo4j, subset in Neo4j, Examples of Neo4j, Applications of Neo4j, Predicate options in Neo4j, Aura for Neo4j, Attribution in Neo4j, Using Neo4j for data visualization, Exploring datasets with Neo4j, Programmatic use in Neo4j, Advanced Neo4j use case, Generating answers with Neo4j, Movies, persons, places in Neo4j, Association in Neo4j, Questions about Neo4j Dash, query Neo4j, Connect in Neo4j, Deprecations in Neo4j, Cache Neo4j data, Query Neo4j, Creating Neo4j index, Connection URI in Neo4j, URI specification in Neo4j, ""neo4j"", Local Neo4j database dump, Triple stores to Neo4j, Procedure call in Neo4j, procedure usage in Neo4j, Postgres and Neo4j, llmNeo4j data training, Fetching information from Neo4j database, DBMS in Neo4j, Databases in Neo4j, Write relationship to Neo4j, Data retrieval from Neo4j, Retrieved information from Neo4j, Writing a book on Neo4j, Books on Neo4j and graphs, Playing with Neo4j code, Toy examples in Neo4j, Document retrieval with Neo4j, parameter passing in Neo4j, API parameters in Neo4j, Operational scenario in Neo4j, NLP with Neo4j, OWL classes in Neo4j, Using Neo4j notebooks, Writing performance in Neo4j, Presenting results in Neo4j, Intelligent results with Neo4j, Acted in relationship in Neo4j, Endless options in Neo4j, entity extraction in Neo4j, NLP in Neo4j, synonyms in Neo4j, applying Neo4j in various domains, implementations in Neo4j, ""Broader than"" relationship in Neo4j, narrower category in Neo4j, open source Neo4j, list of nodes in Neo

In [115]:
gamma_tests[64.0]

,themeCount,themes
0,144,"[Multiple node labels, Projecting multiple node labels, multiple node labels, New syntax for node labels, clone nodes with labels, New nodes with labels, Single node label, Multiple labels per node, another-node-label-value, Various node labels, Multiple labels on nodes, Number of node labels, Specific node labels, TargetNodeLabels, Matching node by labels, Person node label, Entity node label, Person node example, Optional node label, PLACE node label, Patient node label, Single string node label, Using node labels, Projected node label, Nodes with specific labels, disjoint node labels, One label per node, multiple labels on nodes, Train with multiple node labels, Adds labels to nodes, Set node labels, sets labels to nodes, Create node labels, Target node labels, Write node labels to database, returns node labels, Specifying Sequences of node labels, return node labels, Adding node labels dynamically, remove node labels, Creates node labels, Enables node label creation, labels node labels, Label every node, virtual node labels, Rename node label, Character node label, Change node labels, Renaming node labels, Adds new node label, Assigning second node label, Mutate Node Label, Add node label, Get existing node label, Source node label, retain node labels, Remove labels from a node, using node labels, excluding node labels, Filtering on node labels, specify node label, Fast Node-Counts by Label, readLabels for node labels, Hardcoded node labels, Airport node label, :Club node label, :League node label, Given labels for termination nodes, exclude nodes with label, Remove Node Labels, Excluding nodes without specific label, Artist label nodes, Labels per node, Series mapping node label, map of label to nodes, Return node labels list, Dynamic node labeling, Specify node labels in queries, Dedicated node label, Selecting node labels, transform node labels, Graph node labels, Get node labels, preserves node label distribution, Create a new node label, omitting nodes without labels, default all node labels, Page node label, 'Page' node label, Remove node labels, Returns labels of a node, remove labels, Person nodes existence, Labeled node pairs, ""Valid"" node label, One node label ""N"", SourceNodeLabel, List of node label strings, Engineering label in nodes, List of node labels, ...]"
1,137,"[GDS preserves parallel relationships, GDS aggregate parallel relationships, GDS schema, GDS client, GDS license key, GDS support, GDS usage, Python client for GDS, GDS application, GDS users, GDS Community Edition, GDS software, Server-side GDS, server-side GDS, GDS library version 2.0, Getting Started with GDS Client, Enable GDS, Invalid GDS procedures, No GDS algorithms, Running GDS, gds prefix, Sending messages in GDS, GDS, Benefits of GDS, Current GDS workload, Using GDS, Users familiar with GDS, Access GDS resources, GDS offers, GDS Flight server, Load data into GDS, using GDS, Overview of GDS, Manager's Guide to GDS, GDS documentation, GDS client connection, GDS implementation, Data into GDS server, GDS API, GDS version 2.2+, GDS procedure naming, Training methods in GDS, GDS compatibility, GDS beta pipeline, Common usage patterns in GDS, GDS new approach, GDS algorithms completion, GDS workloads, GDS client connection setup, Similarity in GDS, Cosine similarity in GDS, GDS enabled feature, Limitations on community edition, GDS community edition limitations, GDS integration, GDS workflow, GDS representation, GDS node property types, GDS performance optimization, GDS autotuning capabilities, GDS algorithms, GDS Algorithm Performance, GDS logging capabilities, GDS in Bloom, GDS procedure API, GDS beta functions, GDS alpha pipeline, GDS in-memory state, GDS algorithms execution modes, Graph Data Science (GDS) settings, GDS Node Similarity Algorithms, GDS software license, GDS administrator, GDS server enterprise license, Running GDS procedures, ongoingGdsProcedures, GDS-focused analytic workloads, GDS resources, G

In [116]:
gamma_tests[128.0]

,themeCount,themes
0,82,"[Using labels, dynamic labels, dynamic labels creation, yield label, dynamic secondary labels, Different labels, Dynamic labels, dynamic label removal, Label names, Indexed labels, status labels, Edge labels, Graph labels, Set Labels, Config parameters for labels, Specifying labels, determining labels, Create new labels, Custom labels, Using labels in queries, Indexed label, Adjacent labeling, Binary label, Specified label, Specific label usage, Specific label requirement, Distribution of labels, Distribution of tags, Remove Labels, List of labels, Multiple labels support, True labels, Using labels in models, Product labels, contains all labels, SET any label, Apply labels later, Identifier for labels, Data modeling with labels, Use labels for semantic class, Overloading labels, Labeling classes, Data labeling, Applicable to multiple labels, Applied to multiple labels, set of labels, initial labels setup, unique label, Labels limit, Remove labels, Labels, Combination of labels, Multiple labels handling, Unique label, unique label assignment, Species label, sets given labels, Multiple labels, Manually labeled data, Specifying terminator labels, Specifying end labels, Specified labels, Dynamically specified labels, Dynamic label updating, Escaping dynamic labels, database labels, Labels in records, List available labels, Specific label query, Specified labels in query, Correct label combination usage, Selection of labels, Multiple Labels/Types handling, Labels removed, Label management, Graph labeling, Specify field for labels, Label specification, Projecting labels, Projected labels as keys, Labeling, Labels addition]"
1,75,"[Set of nodes, Unique nodes, Distinct nodes, differences between nodes, Distinguishing node types, Different node types, information about nodes, distinct nodes, Node nodes, returns nodes, Limited information on nodes, Components of nodes, processed nodes, Returns nodes individually, nodes returned individually, Intermediate nodes, Section nodes, Tag nodes, classified nodes, DISTINCT Nodes, Green nodes, Stage nodes, k unique nodes, 'k' nodes, Original nodes, Standin nodes, handful nodes, Sparse nodes, sparse nodes, Specific nodes, yield nodes, Three nodes, end nodes, nodes created, end nodes uniqueness, super nodes, relevant nodes, Candidate nodes, Primary nodes, Identifier nodes, Virtual Nodes, Indexed nodes, Parent nodes, Anomalous Nodes, ID uniqueness across nodes, Adjacent nodes, collapse intermediate nodes, intermediate node ids, Device nodes, Entity nodes, Return only one node, Return all nodes, Uniquely identifiable nodes, End nodes uniqueness, Patient nodes, Objects as nodes, Nodes created, Product nodes, Student nodes, Returning nodes, Classification of nodes, Adjacent nodes set, Updating nodes, returning nodes, restrict nodes returned, (:Patient) node, fetches all nodes, Algorithm distinguishes node types, Subject nodes, Returns all nodes in graph, Referenced node objects, Methods of node object, Low-degree nodes, endNodes, skip intermediate nodes]"
2,73,"[GDS preserves parallel relationships, GDS aggregate parallel relationships, GDS schema, GDS client, GDS support, GDS usage, Python client for GDS, GDS application, GDS users, GDS software, Getting Started with GDS Client, GDS, Users familiar with GDS, GDS offers, Manager's Guide to GDS, GDS documentation, GDS client connection, GDS implementation, Data into GDS server, GDS API, GDS version 2.2+, GDS procedure naming, GDS compatibility, GDS beta pipeline, Common usage patterns in GDS, GDS new approach, GDS algorithms completion, GDS write procedures, GDS workloads, GDS enabled feature, GDS integration, GDS workflow, GDS representation, GDS node property types, GDS performance optimization, GDS autotuning capabilities, GDS logging capabilities, GDS procedure API, GDS beta functions, GDS alpha pipeline, GDS in-memory state, GDS administrator, GDS server enterprise license, GDS resources, GDS CE usage, Core operations in GDS

In [117]:
gamma_tests[256.0]

,themeCount,themes
0,51,"[Ontology Domain Model, Ontology model, Ontology development, Ontology description, Ontology creation, Ontology mapping, Ontology integration, Ontology in data science, Ontology definition, Ontology properties, Ontology reference and format, Ontology learning from data, Ontology overlay, Ontology Based Reasoning, Ontology-driven behavior, Ontology, Ontology generation, Ontology learning, Ontology in data, Ontology importation, Ontology filter, Ontology changes, Ontology content display, Ontology level, Ontology connection, Ontology and taxonomy, Ontology definitions, Reading Ontology, Ontology parsing, Deriving ontology, Ontology training, Classes and properties in Ontology, Relationships in Ontology, Ontology relationships, External ontologies integration, Person ontology, Ontologies creation, Ontology loading, Ontology updates, Ontology trimming, Ontology in RDF, Ontology as a canonical model, Ontological models, Department level ontologies, Best explanation of ontology, Ontologies from GraphQL, Ontology-guided approach, Semantic definitions in ontology, Ontology in Cypher, Ontology DB in movies database, Saving and updating ontology]"
1,49,"[apoc.any.property function, apoc.coll function, apoc.coll.avg function, apoc.coll.indexOf function, apoc.coll methods, apoc.coll.combinations function, apoc.coll.pairs, apoc.coll.occurrences function, apoc.coll.different function, apoc.coll.stdev function, apoc.coll.zip function, apoc.coll, apoc.coll.elements function, apoc.coll.toSet function, apoc.coll.union function, apoc.coll.elements, apoc.coll.fill function, apoc.coll.reverse function, apoc.coll.frequencies function, apoc.coll.insert function, apoc.coll.min function, apoc.coll.remove function, apoc.coll.indexOf, apoc.coll.insertAll function, apoc.coll.max function, apoc.coll.sum function, apoc.coll.min, apoc.coll.max, apoc.coll.split, apoc.coll namespace, apoc.coll.union, apoc.coll.disjunction, apoc.coll.fill, apoc.coll.stdev, apoc.coll.remove, apoc.coll.elements query, apoc.agg, apoc.coll.occurrences, apoc.coll.insertAll, apoc.coll.insert, apoc.coll.frequencies, apoc.coll.avg, apoc.coll.combinations, apoc.coll.sum, apoc.coll.sort(), apoc.neighbors function, apoc.coll.duplicates, apoc.coll.elements Procedure, apoc library usage]"
2,45,"[Machine learning pipelines, Machine learning process, Machine learning applications, Applying machine learning, Machine learning capabilities, Machine learning, Machine learning programs, Machine Learning Tasks, machine learning in data science, machine learning use cases, Machine learning focus, Machine learning based method, Machine learning task, Machine learning classification, Traditional machine learning, Machine learning model training, Testing machine learning model, Machine learning setup, Train machine learning model, Machine learning model preparation, machine learning models, Training machine learning models, Machine learning model, Machine learning models, Machine Learning models, Machine learning model selection, Simulate machine learning models, Applying machine learning models, Machine learning expertise, Sharing machine learning models, running machine learning experiments, Training NLP models, Supported machine learning task, Machine learning tasks, Machine Learning, machine learning (ML), Machine learning algorithms, Supervised machine learning model, Machine learning model features, Deep learning models, machine learning model re-training, supervised machine learning, Unsupervised machine learning tasks, Machine Learning libraries, Binary classification]"
3,44,"[LLM enhanced applications, LLMs issues, LLM limitations, LLMs Limitations, LLMs, LLMs definition, LLM capabilities, Local LLMs, Instruction following in LLM, Limitations of LLMs, Defining LLM, LLM model, LLM parameter, LLM space, LLM Agents, LLM-powered applications, LLMs Integration, LLMs assistance, LLMs capabilities, LLM fine-tuning, LLM providers, LLM models, LLM response, LLM, LLM prompts, L

In [118]:
gamma_tests[512.0]

,themeCount,themes
0,35,"[Persistent Volume Claims (PVC), Persistent volume, persistent volume, Creating persistent volumes, persistent volumes, Persistent volumes, Persistent Volumes (PV) in Kubernetes, Provision persistent volumes dynamically, Persistent volume configuration, configure persistent volume, Persistent volume disks, Persistent volume selector, Persistent Volume, Persistent volume for plugins, Persistent volume provisioning, Persistent Volume (PV), PersistentVolume in Kubernetes, Persistent volume claim (PVC), Persistent Volume Selector, Kubernetes volume object, Kubernetes volume types, Mounted volume management, PersistentVolumeClaim object, Persistent Volume Claim, Persistent Disk creation, Persistent volume creation, Persistent volume selection, Persistent volume selector file, Reuse a persistent volume, Persistent volumes in Kubernetes, Persisting data using Volumes, Dynamically provisioned Persistent Volume, persistent volume for backups, Delete persistent volume claims, invoke volume]"
1,30,"[Graph algorithms, graph algorithms, Graph algorithms for disambiguation, Graph Algorithms, Graph algorithm results, Graph algorithm implementation, Graph algorithm execution, Graph algorithm output, Graph algorithms documentation, Graph algorithms for risk measurement, Graph Algorithms reference, Graph algorithm applications, Graph algorithm, Traversal algorithms, Graph algorithms use, Graph algorithms analysis, Graph algorithms usage, Graph coloring algorithms, Graph algorithms in supply chain, Compute algorithms on entire graph, graph algorithms for predictions, Graph algorithm parameters, graph algorithms execution, Graph algorithms guide, Graph algorithm configuration, graph algorithm parameters, graph algorithm timing, Deprecated Graph Algorithms library, Graph problem solving, Algorithm execution on large graphs]"
2,30,"[Docker, Docker official documentation, Docker container, Docker integration, Docker best practices, Docker tags, Docker containers management, Docker specific operations, Docker container integration, Docker containers, Docker orchestration, Docker usage, Uses Docker, Docker user permissions, Docker-specific operations, Docker documentation, Docker maintenance operations, Docker daemon restart, Docker healthcheck, Docker volumes, Docker compose, Using Docker services, Dockerfile, Docker container security, Load docker image, Local deployment via Docker, Introduction to Docker, Docker Run options, Docker run command, Docker-compose command]"
3,29,"[Configuring the pipeline, Configurable pipeline, configuring the pipeline, Creating a pipeline, configurable pipeline, Configure pipeline, Create pipeline syntax, Created pipeline, Creating pipelines, create pipeline, configure pipeline, creating pipelines, reconfigured pipeline splitting, listing pipelines, Created pipeline name, Add feature to pipeline, Creation of data pipelines, configuring a pipeline, Build pipeline, Dropping pipelines, removing pipelines, List pipelines, training the pipeline, List pipelines syntax, Generic pipeline creation, Configuring pipelines, Listing all pipelines, Listing specific pipeline, Building prediction pipeline]"
4,29,"[Nodes with labels, Nodes with multiple labels, Node with multiple labels, multi-labeled nodes, Graph nodes labeling, nodes with multiple labels, Returning nodes with label, Searching for nodes with label, nodes with Country label, nodes with specific label, Nodes with label Message, nodes without a label, Multi-label nodes, nodes with text data, Node with labels :A :B, NODES label, Fetch nodes by label, Nodes with label, dual-labeled nodes, nodes with label MyCountLabel, nodes with Book label, nodes with Actor label, nodes with Author label, Nodes with Movie label, Nodes with :Product label, Node with label: Artist, Disjoint node labels, Connect nodes with label A, nodes labeled A]"
5,29,"[Analyzer configuration, streamNodeProperties configuration, Override configuration, Syntax configuration, names

# See what topics drop out from a cluster at various gamma values

In [119]:
def get_cluster_for_theme(property_name, topic):
    topic_df = gds.graph.nodeProperties.stream(g_topics2,
                                               [property_name],
                                               db_node_properties = ["description"],
                                               separate_property_columns=True
                                              )
    topic_cluster_id = topic_df.loc[topic_df['description']==topic, property_name].iloc[0]
    return topic_df.loc[topic_df[property_name]==topic_cluster_id, "description"].to_list()

In [120]:
optional_match_64 = get_cluster_for_theme("leidenGamma64.0", "Optional MATCH in subqueries")
optional_match_64

['subquery execution',
 'nested subquery',
 'top-level subquery',
 'count subqueries',
 'Correlated subquery',
 'Sub-queries nesting',
 'Subquery expressions',
 'Scope in subqueries',
 'MATCH keyword optional in subqueries',
 'Accessing variables in subqueries',
 'COUNT subqueries',
 'Optional MATCH in subqueries',
 'Restrictions on subqueries',
 'Top-level subquery',
 'existential subquery',
 'COLLECT subquery',
 'First clause of a subquery',
 'reference variables in subqueries',
 'Subquery execution',
 'Subquery evaluation',
 'Subqueries execution order',
 'Subquery executions',
 'Subquery execution failure',
 'Subqueries execution',
 'Execute subqueries',
 'Unit subqueries',
 'Returning subqueries',
 'unit subqueries',
 'Variable reference in subqueries',
 'Evaluating subqueries',
 'Unit subquery',
 'CALL subquery restrictions',
 'Subquery modification',
 'Evaluates subquery',
 'Nested subqueries',
 'Correlated subqueries',
 'Nesting EXISTS subqueries',
 'Subquery variables',
 'Subq

In [121]:
optional_match_128 = get_cluster_for_theme("leidenGamma128.0", "Optional MATCH in subqueries")
[topic for topic in optional_match_64 if topic not in optional_match_128] 

['subquery execution',
 'Subquery expressions',
 'Top-level subquery',
 'Subquery execution',
 'Subquery evaluation',
 'Subqueries execution order',
 'Subquery executions',
 'Subquery execution failure',
 'Subqueries execution',
 'Execute subqueries',
 'Returning subqueries',
 'unit subqueries',
 'Subquery modification',
 'Subquery variables',
 'Subquery RETURN clauses',
 'CALL subquery differences',
 'COLLECT subqueries']

In [122]:
optional_match_256 = get_cluster_for_theme("leidenGamma256.0", "Optional MATCH in subqueries")
[topic for topic in optional_match_128 if topic not in optional_match_256] 

[]

In [123]:
optional_match_512 = get_cluster_for_theme("leidenGamma512.0", "Optional MATCH in subqueries")
[topic for topic in optional_match_256 if topic not in optional_match_512] 

[]

In [124]:
optional_match_512

['nested subquery',
 'top-level subquery',
 'count subqueries',
 'Correlated subquery',
 'Sub-queries nesting',
 'Scope in subqueries',
 'MATCH keyword optional in subqueries',
 'Accessing variables in subqueries',
 'COUNT subqueries',
 'Optional MATCH in subqueries',
 'Restrictions on subqueries',
 'existential subquery',
 'COLLECT subquery',
 'First clause of a subquery',
 'reference variables in subqueries',
 'Unit subqueries',
 'Variable reference in subqueries',
 'Evaluating subqueries',
 'Unit subquery',
 'CALL subquery restrictions',
 'Evaluates subquery',
 'Nested subqueries',
 'Correlated subqueries',
 'Nesting EXISTS subqueries']

In [125]:
backup_128 = get_cluster_for_theme("leidenGamma128.0", "Backup management")
backup_128

['Backup scheduling',
 'Backup modes',
 'Backup options',
 'Backup planning',
 'Backup configuration',
 'Backup server',
 'Backup client',
 'Remote server backup',
 'backups configuration',
 'Backup volume',
 'Backup and restore strategy',
 'backup artifacts',
 'Backup process settings',
 'Backup enabled setting',
 'backup models and graphs',
 'Backup management',
 'backup status',
 'Backup parameters',
 'backup configuration',
 'Backup restoration',
 'Backup support',
 'backup data',
 'Backup failure handling',
 'Backup success',
 'Backup problems',
 'Broken backup risk',
 'Backup run',
 'Backup failure',
 'Backup command behavior',
 'Backup clients',
 'Backup',
 'Backup strategies',
 'Data restoration',
 'Restoring data',
 'Data restoration check']

In [126]:
backup_256 = get_cluster_for_theme("leidenGamma256.0", "Backup management")
[topic for topic in backup_128 if topic not in backup_256] 

['Backup and restore strategy',
 'Backup restoration',
 'Data restoration',
 'Restoring data',
 'Data restoration check']

In [127]:
backup_512 = get_cluster_for_theme("leidenGamma512.0", "Backup management")
[topic for topic in backup_256 if topic not in backup_512] 

['Backup clients', 'Backup server', 'Backup client', 'Remote server backup']

In [128]:
backup_512

['Backup scheduling',
 'Backup modes',
 'Backup options',
 'Backup planning',
 'Backup configuration',
 'backups configuration',
 'Backup volume',
 'backup artifacts',
 'Backup process settings',
 'Backup enabled setting',
 'backup models and graphs',
 'Backup management',
 'backup status',
 'backup data',
 'Backup failure handling',
 'Backup success',
 'Backup problems',
 'Broken backup risk',
 'Backup run',
 'Backup failure',
 'Backup command behavior',
 'Backup',
 'Backup strategies',
 'Backup parameters',
 'backup configuration',
 'Backup support']

# Write Leiden results to Neo4j

In [129]:
gds.set_database("neo4j")

In [130]:
gds.graph.nodeProperties.write(g_topics2, {"leidenGamma512.0": "leidenId"})

writeMillis                      478
graphName            reweight_topics
nodeProperties            [leidenId]
propertiesWritten              38226
Name: 0, dtype: object

In [131]:
gds.run_cypher("""CREATE CONSTRAINT topic_group_node_key IF NOT EXISTS FOR (n:TopicGroup) REQUIRE n.id IS NODE KEY""")

""


In [132]:
gds.run_cypher("""
MATCH (t:Topic)
WHERE t.leidenId is not null
WITH t ORDER BY t.description
WITH t.leidenId AS id, collect(t) AS topicNodes, collect(t.description) AS descriptions
MERGE (g:TopicGroup {id:id}) 
SET g.descriptions = descriptions
WITH g, topicNodes
FOREACH (t in topicNodes | MERGE (t)-[:IN_GROUP]->(g))
""")

""


In [133]:
gds.run_cypher("""
MATCH (g:TopicGroup)
WITH max(g.id) AS maxId
MATCH (t:Topic)
WHERE NOT EXISTS {(t)-[:IN_GROUP]->()}
AND EXISTS {()-[:HAS_TOPIC {prompt:"v2"}]->(t)}
WITH maxId, COLLECT(t) AS needsGroup
UNWIND range(0, size(needsGroup)-1) AS idx
WITH maxId, idx, needsGroup[idx] AS t
MERGE (g:TopicGroup {id:maxId + idx + 1})
SET g.descriptions = [t.description]
WITH g, t
MERGE (t)-[:IN_GROUP]->(g)
""")

""


# Clean up

In [134]:
gds.graph.drop(g_topics)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                topics
database                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [135]:
gds.graph.drop(g_topics2)

graphName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       reweight_topics
database                                                                                                                                                